In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from zipfile import ZipFile
file_name = "/content/drive/My Drive/Covid-19 data.zip"
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print("Completed")

Completed


In [3]:
from tensorflow.keras.layers import *  # here * means every layer
from tensorflow.keras.models import * 
import tensorflow.keras as tf

In [4]:
#Training model
model = Sequential()   # creating a blank model 
                       # here we are importing layers also so we can directly pass Sequential()
model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(224,224,3))) # it is convolutional 2D layer
# 32 is number of feature map
# the images I am going to pass it will be 224X224 pixels and 3 means RGB image 
# we are passing the colour image
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))    # reduce the overfitting
 
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
 
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
 
model.add(Flatten())    # input layer

#ANN 
model.add(Dense(64,activation='relu'))    # hidden layer of ann
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))   # output layer

# Compile the model 
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [5]:
#Moulding train images
from tensorflow.keras.preprocessing import image
train_datagen = image.ImageDataGenerator(rescale = 1./255, shear_range = 0.2,zoom_range = 0.2, horizontal_flip = True)
 
test_dataset = image.ImageDataGenerator(rescale=1./255)

In [6]:
#Reshaping test and validation images 
train_generator = train_datagen.flow_from_directory(
    '/content/Covid-19 data/Train',
    target_size = (224,224),
    batch_size = 32, 
    class_mode = 'binary')
validation_generator = test_dataset.flow_from_directory(
    '/content/Covid-19 data/Val',
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'binary')

Found 5120 images belonging to 2 classes.
Found 112 images belonging to 2 classes.


In [9]:
#### Train the model
history = model.fit_generator(
    train_generator,
    steps_per_epoch=160,
    epochs = 10,
    validation_data = validation_generator,
    validation_steps=1
)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
160/160 [==============================] - 112s 698ms/step - loss: 0.3061 - accuracy: 0.8621 - val_loss: 0.4669 - val_accuracy: 0.8438
Epoch 2/10
160/160 [==============================] - 111s 695ms/step - loss: 0.2718 - accuracy: 0.8867 - val_loss: 0.3597 - val_accuracy: 0.8438
Epoch 3/10
160/160 [==============================] - 111s 695ms/step - loss: 0.2437 - accuracy: 0.9035 - val_loss: 0.2738 - val_accuracy: 0.9375
Epoch 4/10
160/160 [==============================] - 110s 689ms/step - loss: 0.2187 - accuracy: 0.9170 - val_loss: 0.2998 - val_accuracy: 0.8750
Epoch 5/10
160/160 [==============================] - 110s 686ms/step - loss: 0.2069 - accuracy: 0.9207 - val_loss: 0.2590 - val_accuracy: 0.8125
Epoch 6/10
160/160 [==============================] - 117s 731ms/step - loss: 0.2185 - accuracy: 0.9199 - val_loss: 0.2533 - val_accuracy: 0.8750
Epoch 7/10
160/160 [==============================] - 120s 749ms/step - loss: 0.1927 - accuracy: 0.9309 - val_loss: 0.1187 -

In [20]:
from tensorflow.keras.preprocessing import image
import numpy as np
img = image.load_img('/content/Normal.jpeg',target_size=(224,224))
img = image.img_to_array(img)
img = np.expand_dims(img,axis=0)   ### flattening
ypred = model.predict(img)
if ypred[0][0] == 1:
  print("Covid Negative")
else:
  print("Covid Positive")

Covid Negative


In [11]:
ypred[0][0]

0.0

In [12]:
#### save the model
model.save("covid_model.h5")

In [13]:
import tensorflow.keras as tf
mymodel = tf.models.load_model("/content/covid_model.h5")

In [14]:
from tensorflow.keras.preprocessing import image
import numpy as np
img = image.load_img('/content/covid.jpg',target_size=(224,224))
img = image.img_to_array(img)
img = np.expand_dims(img,axis=0)   ### flattening
ypred = mymodel.predict(img)
if ypred[0][0] == 1:
  print("Covid Negative")
else:
  print("Covid Positive")

Covid Positive
